In [ ]:
# Librerías
import requests
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from astroquery.mpc import MPC

# Módulos 
from math import ceil

# Cobs API

In [2]:
# Verificar la conexión a internet.
def verificar_conexion():
    try:
        requests.get("http://www.google.com", timeout=5)
        print('✅ Conectado a internet.')
        return True
    
    except requests.ConnectionError:
        print('🛑 Sin conexión a internet.')
        return False

In [143]:
# Conexión con la API de COBS
try:
    content = [] 
    fecha_inicial = '1976-01-01 00:00'
    nombre_cometa = 'C/2023 A3'
    Link_cops_API_pagina_1 = f'https://cobs.si/api/obs_list.api?des={nombre_cometa}&format=json&from_date={fecha_inicial}&page=1&exclude_faint=False&exclude_not_accurate=False'

    if True:
        print(f'\n⌛ Conectando con la base de datos [COBS Observaciones].')
        response_pagina_1 = requests.get(Link_cops_API_pagina_1)

        if response_pagina_1.status_code == 200:
            content_pagina_1 = response_pagina_1.json()
            numero_de_paginas = int(content_pagina_1['info']['pages'])

            content.extend(content_pagina_1['objects'])

            for pagina in range(2, numero_de_paginas + 1):
                Link_cops_API_pagina = f'https://cobs.si/api/obs_list.api?des={nombre_cometa}&format=json&from_date={fecha_inicial}&page={pagina}&exclude_faint=False&exclude_not_accurate=False'
                response_pagina = requests.get(Link_cops_API_pagina)
                content_pagina = response_pagina.json()
                content.extend(content_pagina['objects'])

            print('✅ Base de datos actualizada [COBS Observaciones].')
        
except requests.ConnectionError:
    print(f'🛑 Se presentó un error al cargar la base de datos.\nError: {response_pagina.status_code}\n{response_pagina.content}')


⌛ Conectando con la base de datos [COBS Observaciones].
✅ Base de datos actualizada [COBS Observaciones].


In [144]:
# Creación del data frame Cometa
cometa_df = pd.DataFrame(content)
cometa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2426 entries, 0 to 2425
Data columns (total 47 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   type                       2426 non-null   object 
 1   obs_date                   2426 non-null   object 
 2   comet                      2426 non-null   object 
 3   observer                   2426 non-null   object 
 4   location                   1252 non-null   object 
 5   extinction                 331 non-null    object 
 6   obs_method                 2425 non-null   object 
 7   comet_visibility           21 non-null     object 
 8   magnitude                  2424 non-null   object 
 9   conditions                 114 non-null    object 
 10  ref_catalog                2425 non-null   object 
 11  instrument_aperture        2426 non-null   object 
 12  instrument_type            2426 non-null   object 
 13  instrument_focal_ratio     1543 non-null   float

In [145]:
# Numero de registros y variables sin filtrar la información
filas,columnas = cometa_df.shape
print(f'Registros: {filas}\nVariables: {columnas}')

Registros: 2426
Variables: 47


In [146]:
# Base de datos arrojada por la API
cometa_df.sample(5)

,type,obs_date,comet,observer,location,extinction,obs_method,comet_visibility,magnitude,conditions,...,magnitude_error,comparison_star_magnitude,pixel_size_x,pixel_size_y,pixel_size_unit,obs_comment,obs_sky_quality,obs_sky_quality_method,reference_star_names,date_added
1371,V,2024-07-01 22:04:48,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Juan Jose', 'last_name': 'Gonz...",None,None,"{'key': 'S', 'name': 'In-Out method', 'source'...",None,9.9,None,...,None,None,None,None,None,,None,NaN,,2024-07-02 11:09:23
1587,C,2024-05-29 12:57:35,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Katsumi', 'last_name': 'Yoshim...",None,None,"{'key': 'D', 'name': 'Johnson/Bessel/Kron/Cous...",None,11.5,None,...,None,12.30,2.0,2.0,s,,None,NaN,,2024-05-31 01:19:57
411,V,2024-10-28 18:50:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Tibor', 'last_name': 'Csorgei'...",Bratislava,None,"{'key': 'O', 'name': 'Out-of-focus (or extrafo...",None,6.0,None,...,None,None,None,None,None,,None,NaN,,2024-10-28 18:53:30
1632,C,2024-05-23 15:00:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Jiarui', 'last_name': 'Xiong',...","Daocheng, China(remote,O40)",None,"{'key': 'C', 'name': 'Unfiltered total CCD mag...",None,10.7,None,...,None,None,None,None,None,,None,NaN,,2024-05-23 15:07:58
541,V,2024-10-23 17:47:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Christian', 'last_name': 'Hard...",Jeersdorf f,None,"{'key': 'S', 'name': 'In-Out method', 'source'...",None,5.5,None,...,None,None,None,None,None,die Sonne stand 15° unter dem Horizont. Der K...,19.5,3.0,,2024-10-23 20:42:15


In [147]:
# Métodos de observación
cometa_df.obs_method.apply(
    lambda registro: f"{registro['key']}: {registro['name']}" if (registro is not None) and ('key' in registro) and ('name' in registro) else 'Datos faltantes'
).value_counts()

obs_method
C: Unfiltered total CCD magnitude                                                   583
M: Modified-Out method                                                              516
S: In-Out method                                                                    424
Z: CCD Visual equivalent                                                            347
B: Simple Out-Out method                                                            157
V: Johnson/Bessel/Kron/Cousins V with CCD                                           108
I: In-focus                                                                         104
k: Kron/Cousins R with CCD                                                           46
D: Johnson/Bessel/Kron/Cousins B with CCD                                            27
H: Kron/Cousins I with CCD                                                           21
-: Unknown                                                                           18
P: photographic      

In [148]:
# Tratamiento de los datos de interés
cometa_df['obs_method_key'] = cometa_df.obs_method.apply(lambda registro: registro['key'] if registro is not None and 'key' in registro else 'Dato faltante')
cometa_df['obs_date'] = pd.to_datetime(pd.to_datetime(cometa_df.obs_date).dt.date)
cometa_df['magnitude'] = pd.to_numeric(cometa_df.magnitude)

In [149]:
# Creación del data frame curva de luz cruda
curva_de_luz_cruda_df = cometa_df[['obs_method_key', 'obs_date', 'magnitude']].copy()
curva_de_luz_cruda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2426 entries, 0 to 2425
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  2426 non-null   object        
 1   obs_date        2426 non-null   datetime64[ns]
 2   magnitude       2424 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 57.0+ KB


In [150]:
# Numero de registros y variables con la información filtrada
filas,columnas = curva_de_luz_cruda_df.shape
print(f'Registros: {filas}\nVariables: {columnas}')

Registros: 2426
Variables: 3


In [151]:
# Data Frame de la curva de luz
curva_de_luz_cruda_df.sample(5)

,obs_method_key,obs_date,magnitude
1297,Z,2024-07-29,8.9
1997,C,2024-04-19,11.0
2366,C,2023-05-14,16.4
305,M,2024-11-03,6.9
1884,Z,2024-05-01,10.6


In [152]:
# Curva de luz cruda.
labels = {'obs_date':'Observation Date','magnitude':'Apparent total magnitude', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_cruda_df, x='obs_date', y='magnitude', color='obs_method_key', template= 'plotly_dark', labels= labels, title= f'Lightcurve of comet {nombre_cometa}')
fig.update_yaxes(autorange="reversed")
fig.show()

# Perihelio Cobs API

In [153]:
# Conexión con la API de COBS para obtener el perihelio
try: 
    Link_cops_API = f'https://cobs.si/api/comet.api?des={nombre_cometa}'

    if verificar_conexion():
        response = requests.get(Link_cops_API)

        if response.status_code == 200:
            perihelio = pd.to_datetime(response.json()['object']['perihelion_date'])
            print('✅ Perihelio del cometa obtenido.')
    
except requests.ConnectionError:
    print(f'🛑 Se presentó un error al cargar la base de datos.\nError: {response.status_code}\n{response.content}')

✅ Conectado a internet.
✅ Perihelio del cometa obtenido.


# MPC API usando astroquery.

In [ ]:
# Creación de data frame Ephemeris (conexión con la API del MPC)
efemerides_total = []

fecha_inicial = curva_de_luz_cruda_df.obs_date.min()
fecha_final = curva_de_luz_cruda_df.obs_date.max()
fechas = (fecha_final - fecha_inicial).days + 1

print('⌛ Conectando con la base de datos [MPC efemerides].')
for i in range(ceil(fechas/1441)):
    efemerides = MPC.get_ephemeris(nombre_cometa, start = str(fecha_inicial), number = 1441)  # type: ignore
    efemerides_ciclo_df = efemerides.to_pandas()
    efemerides_total.append(efemerides_ciclo_df)

    fecha_inicial = efemerides_ciclo_df.Date.max()

# Creación del data frame efemerides filtrada
efemerides_df = pd.concat(efemerides_total)
efemerides_df.columns = efemerides_df.columns.str.lower().str.replace(' ', '_')

efemerides_filtrada_df = efemerides_df[['date', 'delta','r', 'phase']].copy()
efemerides_filtrada_df = efemerides_filtrada_df.rename(columns = {'date':'obs_date'})
efemerides_filtrada_df['obs_date'] = pd.to_datetime(pd.to_datetime(efemerides_filtrada_df.obs_date).dt.date)
efemerides_filtrada_df.reset_index(inplace = True)

efemerides_filtrada_df

,date,ra,dec,delta,r,elongation,phase,v,proper_motion,direction
0,2023-02-25,230.565417,-1.256111,6.946,7.290,106.6,7.5,17.6,8.17,318.9
1,2023-02-26,230.528333,-1.214722,6.921,7.281,107.6,7.4,17.6,8.51,317.4
2,2023-02-27,230.488750,-1.172778,6.896,7.272,108.6,7.4,17.6,8.86,316.0
3,2023-02-28,230.446250,-1.129722,6.872,7.264,109.6,7.4,17.6,9.21,314.7
4,2023-03-01,230.401250,-1.086389,6.847,7.255,110.6,7.3,17.6,9.56,313.4
...,...,...,...,...,...,...,...,...,...,...
656,2024-12-12,290.318333,4.983611,2.264,1.658,41.3,23.1,10.0,43.16,79.3
657,2024-12-13,290.600417,5.037778,2.290,1.674,40.8,22.6,10.1,42.66,78.9
658,2024-12-14,290.878750,5.093056,2.316,1.690,40.3,22.1,10.1,42.19,78.5
659,2024-12-15,291.153750,5.149722,2.342,1.707,39.8,21.7,10.2,41.74,78.1


In [155]:
# Info del data frame ephemeris
ephemeris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 661 entries, 0 to 660
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           661 non-null    datetime64[ns]
 1   ra             661 non-null    float64       
 2   dec            661 non-null    float64       
 3   delta          661 non-null    float64       
 4   r              661 non-null    float64       
 5   elongation     661 non-null    float64       
 6   phase          661 non-null    float64       
 7   v              661 non-null    float64       
 8   proper_motion  661 non-null    float64       
 9   direction      661 non-null    float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 51.8 KB


In [156]:
# Dar a los datos el formato deseado
ephemeris_df.date = pd.to_datetime(ephemeris_df.date)
ephemeris_df.date = pd.to_datetime(ephemeris_df.date.dt.date)
ephemeris_df.dtypes

date             datetime64[ns]
ra                      float64
dec                     float64
delta                   float64
r                       float64
elongation              float64
phase                   float64
v                       float64
proper_motion           float64
direction               float64
dtype: object

In [157]:
# Creación del data frame ephemeris filtrada
ephemeris_filtrada_df = ephemeris_df[['date', 'delta','r', 'phase']].copy()
ephemeris_filtrada_df = ephemeris_filtrada_df.rename(columns = {'date':'obs_date'})
ephemeris_filtrada_df

,obs_date,delta,r,phase
0,2023-02-25,6.946,7.290,7.5
1,2023-02-26,6.921,7.281,7.4
2,2023-02-27,6.896,7.272,7.4
3,2023-02-28,6.872,7.264,7.4
4,2023-03-01,6.847,7.255,7.3
...,...,...,...,...
656,2024-12-12,2.264,1.658,23.1
657,2024-12-13,2.290,1.674,22.6
658,2024-12-14,2.316,1.690,22.1
659,2024-12-15,2.342,1.707,21.7


# Unión de las bases de datos.

In [158]:
# Unión de las bases de datos COBS y MPC
curva_de_luz_procesada_df = curva_de_luz_cruda_df.merge(ephemeris_filtrada_df, on='obs_date')
curva_de_luz_procesada_df

,obs_method_key,obs_date,magnitude,delta,r,phase
0,C,2024-12-15,9.9,2.342,1.707,21.7
1,B,2024-12-13,10.3,2.290,1.674,22.6
2,Z,2024-12-12,9.2,2.264,1.658,23.1
3,S,2024-12-12,9.4,2.264,1.658,23.1
4,S,2024-12-10,9.5,2.211,1.624,24.1
...,...,...,...,...,...,...
2421,C,2023-02-27,18.0,6.896,7.272,7.4
2422,C,2023-02-26,18.1,6.921,7.281,7.4
2423,C,2023-02-25,18.2,6.946,7.290,7.5
2424,C,2023-02-25,17.9,6.946,7.290,7.5


In [159]:
# Información del data frame curva de lus procesada
curva_de_luz_procesada_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2426 entries, 0 to 2425
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  2426 non-null   object        
 1   obs_date        2426 non-null   datetime64[ns]
 2   magnitude       2424 non-null   float64       
 3   delta           2426 non-null   float64       
 4   r               2426 non-null   float64       
 5   phase           2426 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 113.8+ KB


In [160]:
# Reducción de la magnitud aparente y calculo del Delta t
beta = 0

curva_de_luz_procesada_df['magnitud_reducida'] = (
    curva_de_luz_cruda_df['magnitude'] 
    - 5 * np.log10(curva_de_luz_procesada_df['delta'] * curva_de_luz_procesada_df['r'])
    - (beta * curva_de_luz_procesada_df['phase'])
    )

curva_de_luz_procesada_df['delta_t'] = (curva_de_luz_procesada_df.obs_date - perihelio) # type: ignore
curva_de_luz_procesada_df['delta_t'] = curva_de_luz_procesada_df.delta_t.apply(lambda delta_t: delta_t.days)

curva_de_luz_procesada_df

,obs_method_key,obs_date,magnitude,delta,r,phase,magnitud_reducida,delta_t
0,C,2024-12-15,9.9,2.342,1.707,21.7,6.890898,78
1,B,2024-12-13,10.3,2.290,1.674,22.6,7.382045,76
2,Z,2024-12-12,9.2,2.264,1.658,23.1,6.327695,75
3,S,2024-12-12,9.4,2.264,1.658,23.1,6.527695,75
4,S,2024-12-10,9.5,2.211,1.624,24.1,6.724126,73
...,...,...,...,...,...,...,...,...
2421,C,2023-02-27,18.0,6.896,7.272,7.4,9.498744,-579
2422,C,2023-02-26,18.1,6.921,7.281,7.4,9.588201,-580
2423,C,2023-02-25,18.2,6.946,7.290,7.5,9.677688,-581
2424,C,2023-02-25,17.9,6.946,7.290,7.5,9.377688,-581


In [161]:
# Curva de luz reducida
labels = {'obs_date':'Observation Date','magnitud_reducida':'Apparent total magnitude processed', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_procesada_df, x='obs_date', y='magnitud_reducida', color='obs_method_key', template= 'plotly_dark', labels= labels, title=f'Reduced Lightcurve of comet {nombre_cometa}')
fig.update_yaxes(autorange="reversed")
fig.show()

In [162]:
# Curva de luz reducida
labels = {'delta_t':'t-Δt','magnitud_reducida':'Apparent total magnitude processed', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_procesada_df, x='delta_t', y='magnitud_reducida', color='obs_method_key', template= 'plotly_dark', labels= labels, title=f'Reduced Lightcurve of comet {nombre_cometa}')
fig.update_yaxes(autorange="reversed")
fig.show()

In [163]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 9

curva_de_luz_promediada_df = curva_de_luz_procesada_df.copy()
curva_de_luz_promediada_df['promedio_movil'] = curva_de_luz_promediada_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_promediada_df

,obs_method_key,obs_date,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,C,2024-12-15,9.9,2.342,1.707,21.7,6.890898,78,NaN
1,B,2024-12-13,10.3,2.290,1.674,22.6,7.382045,76,NaN
2,Z,2024-12-12,9.2,2.264,1.658,23.1,6.327695,75,NaN
3,S,2024-12-12,9.4,2.264,1.658,23.1,6.527695,75,NaN
4,S,2024-12-10,9.5,2.211,1.624,24.1,6.724126,73,NaN
...,...,...,...,...,...,...,...,...,...
2421,C,2023-02-27,18.0,6.896,7.272,7.4,9.498744,-579,9.492462
2422,C,2023-02-26,18.1,6.921,7.281,7.4,9.588201,-580,9.506261
2423,C,2023-02-25,18.2,6.946,7.290,7.5,9.677688,-581,9.530004
2424,C,2023-02-25,17.9,6.946,7.290,7.5,9.377688,-581,9.511935


In [164]:
# Curva de luz Promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Max apparent total magnitude reduced', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_promediada_df, x='obs_date', y='promedio_movil', color='obs_method_key', template= 'plotly_dark', labels= labels, title= f'Average Lightcurve of comet {nombre_cometa}')
fig.update_yaxes(autorange="reversed")
fig.show()

In [165]:
# Curva de luz Promediada
labels = {'delta_t':'t - Δt','magnitud_reducida':'Max apparent total magnitude reduced', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_promediada_df, x='delta_t', y='promedio_movil', color='obs_method_key', template= 'plotly_dark', labels= labels, title= f'Average Lightcurve of comet {nombre_cometa}')
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz interna (Envolvente inferior) v1 (Promedio corrido -> agrupación)

In [166]:
# Creación del data frame curva de luz agrupada
curva_de_luz_interna_v1_df = curva_de_luz_promediada_df.groupby(by = 'obs_date').max()
curva_de_luz_interna_v1_df = curva_de_luz_interna_v1_df.reset_index()

curva_de_luz_interna_v1_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,2023-02-25,C,18.3,6.946,7.290,7.5,9.777688,-581,9.530004
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,-580,9.506261
2,2023-02-27,C,18.0,6.896,7.272,7.4,9.498744,-579,9.523057
3,2023-02-28,C,18.2,6.872,7.264,7.4,9.708705,-578,9.531429
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,-577,9.528171
...,...,...,...,...,...,...,...,...,...
387,2024-12-09,Z,9.9,2.184,1.608,24.6,7.172307,72,6.681153
388,2024-12-10,Z,9.5,2.211,1.624,24.1,6.724126,73,6.649885
389,2024-12-12,Z,9.4,2.264,1.658,23.1,6.527695,75,NaN
390,2024-12-13,B,10.3,2.290,1.674,22.6,7.382045,76,NaN


In [167]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_v1_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='red', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [168]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_v1_df, x='delta_t', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='red', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz Externa (Envolvente superior) v1 (Promedio corrido -> agrupación)

In [169]:
# Creación del data frame curva de luz agrupada
curva_de_luz_externa_v1_df = curva_de_luz_promediada_df.groupby(by = 'obs_date').min()
curva_de_luz_externa_v1_df = curva_de_luz_externa_v1_df.reset_index()
curva_de_luz_externa_v1_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,2023-02-25,C,17.9,6.946,7.290,7.5,9.377688,-581,9.511935
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,-580,9.506261
2,2023-02-27,C,17.9,6.896,7.272,7.4,9.398744,-579,9.492462
3,2023-02-28,C,17.7,6.872,7.264,7.4,9.208705,-578,9.518689
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,-577,9.528171
...,...,...,...,...,...,...,...,...,...
387,2024-12-09,S,9.2,2.184,1.608,24.6,6.472307,72,6.580071
388,2024-12-10,B,9.1,2.211,1.624,24.1,6.324126,73,6.649885
389,2024-12-12,S,9.2,2.264,1.658,23.1,6.327695,75,NaN
390,2024-12-13,B,10.3,2.290,1.674,22.6,7.382045,76,NaN


In [170]:
# Gráfica de lus promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_v1_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max averaged Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='yellow', size=6, line= dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [171]:
# Gráfica de lus promediada
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_v1_df, x='delta_t', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max averaged Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='yellow', size=6, line= dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz interna (Envolvente inferior) v2 (Agrupación  -> promedio corrido)

In [172]:
# Creación del data frame curva de luz agrupada
curva_de_luz_agrupada_max_v2_df = curva_de_luz_procesada_df.groupby(by = 'obs_date').max()
curva_de_luz_agrupada_max_v2_df = curva_de_luz_agrupada_max_v2_df.reset_index()
curva_de_luz_agrupada_max_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t
0,2023-02-25,C,18.3,6.946,7.290,7.5,9.777688,-581
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,-580
2,2023-02-27,C,18.0,6.896,7.272,7.4,9.498744,-579
3,2023-02-28,C,18.2,6.872,7.264,7.4,9.708705,-578
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,-577
...,...,...,...,...,...,...,...,...
387,2024-12-09,Z,9.9,2.184,1.608,24.6,7.172307,72
388,2024-12-10,Z,9.5,2.211,1.624,24.1,6.724126,73
389,2024-12-12,Z,9.4,2.264,1.658,23.1,6.527695,75
390,2024-12-13,B,10.3,2.290,1.674,22.6,7.382045,76


In [173]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_agrupada_max_v2_df, x='obs_date', y='magnitud_reducida', template= 'plotly_dark',color='obs_method_key', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa}')
fig.update_yaxes(autorange="reversed")
fig.show()

In [174]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 9

curva_de_luz_interna_v2_df = curva_de_luz_agrupada_max_v2_df.copy()
curva_de_luz_interna_v2_df['promedio_movil'] = curva_de_luz_interna_v2_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_interna_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,2023-02-25,C,18.3,6.946,7.290,7.5,9.777688,-581,NaN
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,-580,NaN
2,2023-02-27,C,18.0,6.896,7.272,7.4,9.498744,-579,NaN
3,2023-02-28,C,18.2,6.872,7.264,7.4,9.708705,-578,NaN
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,-577,NaN
...,...,...,...,...,...,...,...,...,...
387,2024-12-09,Z,9.9,2.184,1.608,24.6,7.172307,72,6.859749
388,2024-12-10,Z,9.5,2.211,1.624,24.1,6.724126,73,6.828941
389,2024-12-12,Z,9.4,2.264,1.658,23.1,6.527695,75,6.815959
390,2024-12-13,B,10.3,2.290,1.674,22.6,7.382045,76,6.926522


In [175]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_v2_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='red', size=5, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [176]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_v2_df, x='delta_t', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='red', size=5, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz Externa (Envolvente superior) v2 (Agrupación  -> promedio corrido)

In [177]:
# Creación del data frame curva de luz agrupada
curva_de_luz_agrupada_min_v2_df = curva_de_luz_procesada_df.groupby(by = 'obs_date').min()
curva_de_luz_agrupada_min_v2_df = curva_de_luz_agrupada_min_v2_df.reset_index()
curva_de_luz_agrupada_min_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t
0,2023-02-25,C,17.9,6.946,7.290,7.5,9.377688,-581
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,-580
2,2023-02-27,C,17.9,6.896,7.272,7.4,9.398744,-579
3,2023-02-28,C,17.7,6.872,7.264,7.4,9.208705,-578
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,-577
...,...,...,...,...,...,...,...,...
387,2024-12-09,S,9.2,2.184,1.608,24.6,6.472307,72
388,2024-12-10,B,9.1,2.211,1.624,24.1,6.324126,73
389,2024-12-12,S,9.2,2.264,1.658,23.1,6.327695,75
390,2024-12-13,B,10.3,2.290,1.674,22.6,7.382045,76


In [178]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_agrupada_min_v2_df, x='obs_date', y='magnitud_reducida', template= 'plotly_dark',color='obs_method_key', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa}')
fig.update_yaxes(autorange="reversed")
fig.show()

In [179]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 7

curva_de_luz_externa_v2_df = curva_de_luz_agrupada_min_v2_df.copy()
curva_de_luz_externa_v2_df['promedio_movil'] = curva_de_luz_externa_v2_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_externa_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,2023-02-25,C,17.9,6.946,7.290,7.5,9.377688,-581,NaN
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,-580,NaN
2,2023-02-27,C,17.9,6.896,7.272,7.4,9.398744,-579,NaN
3,2023-02-28,C,17.7,6.872,7.264,7.4,9.208705,-578,NaN
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,-577,NaN
...,...,...,...,...,...,...,...,...,...
387,2024-12-09,S,9.2,2.184,1.608,24.6,6.472307,72,6.413116
388,2024-12-10,B,9.1,2.211,1.624,24.1,6.324126,73,6.461279
389,2024-12-12,S,9.2,2.264,1.658,23.1,6.327695,75,6.432149
390,2024-12-13,B,10.3,2.290,1.674,22.6,7.382045,76,6.575585


In [180]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_v2_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max Averaged Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='yellow', size=5, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [181]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_v2_df, x='delta_t', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max Averaged Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='yellow', size=5, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz usando la mediada v1 (Mediana de cada día registrado).

In [182]:
# Creación del data frame curva de luz mediana v1
curva_de_luz_mediada_v1_df = curva_de_luz_procesada_df.groupby(by= 'obs_date').median(numeric_only= True)
curva_de_luz_mediada_v1_df = curva_de_luz_mediada_v1_df.reset_index()
curva_de_luz_mediada_v1_df

,obs_date,magnitude,delta,r,phase,magnitud_reducida,delta_t
0,2023-02-25,18.20,6.946,7.290,7.5,9.677688,-581.0
1,2023-02-26,18.10,6.921,7.281,7.4,9.588201,-580.0
2,2023-02-27,17.95,6.896,7.272,7.4,9.448744,-579.0
3,2023-02-28,17.95,6.872,7.264,7.4,9.458705,-578.0
4,2023-03-01,17.90,6.847,7.255,7.3,9.419311,-577.0
...,...,...,...,...,...,...,...
387,2024-12-09,9.55,2.184,1.608,24.6,6.822307,72.0
388,2024-12-10,9.40,2.211,1.624,24.1,6.624126,73.0
389,2024-12-12,9.30,2.264,1.658,23.1,6.427695,75.0
390,2024-12-13,10.30,2.290,1.674,22.6,7.382045,76.0


In [183]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_mediada_v1_df, x='obs_date', y='magnitud_reducida', template= 'plotly_dark', labels= labels, title=f'Mediated Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='aquamarine', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [184]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_mediada_v1_df, x='delta_t', y='magnitud_reducida', template= 'plotly_dark', labels= labels, title=f'Mediated Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='aquamarine', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz usando la mediada v2 (Mediana de las dos curvas).

In [185]:
# Creación del data frame curva de luz mediana v2
curva_de_luz_mediada_v2_df = curva_de_luz_externa_v2_df.copy()
curva_de_luz_mediada_v2_df['mediana'] = (curva_de_luz_interna_v2_df['promedio_movil'] + curva_de_luz_externa_v2_df['promedio_movil'])/2
curva_de_luz_mediada_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil,mediana
0,2023-02-25,C,17.9,6.946,7.290,7.5,9.377688,-581,NaN,NaN
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,-580,NaN,NaN
2,2023-02-27,C,17.9,6.896,7.272,7.4,9.398744,-579,NaN,NaN
3,2023-02-28,C,17.7,6.872,7.264,7.4,9.208705,-578,NaN,NaN
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,-577,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
387,2024-12-09,S,9.2,2.184,1.608,24.6,6.472307,72,6.413116,6.636433
388,2024-12-10,B,9.1,2.211,1.624,24.1,6.324126,73,6.461279,6.645110
389,2024-12-12,S,9.2,2.264,1.658,23.1,6.327695,75,6.432149,6.624054
390,2024-12-13,B,10.3,2.290,1.674,22.6,7.382045,76,6.575585,6.751053


In [186]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_mediada_v2_df, x='obs_date', y='mediana', template= 'plotly_dark', labels= labels, title=f'Mediated Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='aquamarine', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [187]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_mediada_v2_df, x='delta_t', y='mediana', template= 'plotly_dark', labels= labels, title=f'Mediated Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='aquamarine', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Comparación de las curvas de luz v1 (Promedio corrido -> agrupación)

In [188]:
# Gráfica de luz promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_v1_df.obs_date, y=curva_de_luz_externa_v1_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v1_df.obs_date, y=curva_de_luz_interna_v1_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_mediada_v1_df.obs_date, y=curva_de_luz_mediada_v1_df.magnitud_reducida, mode='markers', name='Mediana', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
# fig.add_trace(go.Scatter(x=curva_de_luz_externa_v2_df.obs_date, y=curva_de_luz_externa_v2_df.promedio_movil, mode='markers', name='Envolvente_v2', marker=dict(color='green', line=dict(width=1, color='DarkSlateGrey'))))
# fig.add_trace(go.Scatter(x=curva_de_luz_interna_v1_df.obs_date, y=curva_de_luz_interna_v2_df.promedio_movil, mode='markers', name='Núcleo_v2', marker=dict(color='blue', line=dict(width=1, color='DarkSlateGrey'))))
# fig.add_trace(go.Scatter(x=curva_de_luz_mediada_v1_df.obs_date, y=curva_de_luz_mediada_v2_df.mediana, mode='markers', name='Mediana_v2', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='Observation Date', yaxis_title='Averaged Magnitude', title = f'Max/Min Averaged Lightcurve of comet {nombre_cometa}')
fig.show()

In [189]:
# Gráfica de luz promediada
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_v1_df.delta_t, y=curva_de_luz_externa_v1_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v1_df.delta_t, y=curva_de_luz_interna_v1_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_mediada_v1_df.delta_t, y=curva_de_luz_mediada_v1_df.magnitud_reducida, mode='markers', name='Mediana', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='t - Δt', yaxis_title='Averaged Magnitude', title = f'Max/Min Averaged Lightcurve of comet {nombre_cometa}')
fig.show()

# Comparación de las curvas de luz v2 (Agrupación -> promedio corrido)

In [190]:
# Gráfica de luz promediada
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_v2_df.obs_date, y=curva_de_luz_externa_v2_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v2_df.obs_date, y=curva_de_luz_interna_v2_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v2_df.obs_date, y=curva_de_luz_mediada_v2_df.mediana, mode='markers', name='Mediana', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='Observation Date', yaxis_title='Averaged Magnitude', title = f'Max/Min Averaged Lightcurve of comet {nombre_cometa}')
fig.show()

In [191]:
# Gráfica de luz promediada
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_v2_df.delta_t, y=curva_de_luz_externa_v2_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v2_df.delta_t, y=curva_de_luz_interna_v2_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v2_df.delta_t, y=curva_de_luz_mediada_v2_df.mediana, mode='markers', name='Mediana', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))

fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='t - Δt', yaxis_title='Averaged Magnitude', title = f'Max/Min Averaged Lightcurve of comet {nombre_cometa}')
fig.show()